## Creating a Simple MODFLOW 6 Model with Flopy

The purpose of this notebook is to demonstrate the Flopy capabilities for building a simple MODFLOW 6 model from scratch, running the model, and viewing the results.  This notebook will demonstrate the capabilities using a simple well example.  

![mf6](./images/mf6_flowchart.png)

### Setup the Notebook Environment

In [32]:
import os

In [ ]:
import sys
from tempfile import TemporaryDirectory

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import flopy

print(sys.version)
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(mpl.__version__))
print("flopy version: {}".format(flopy.__version__))

In [34]:
# For this example, we will set up a temporary workspace.
# Model input files and output files will reside here.
temp_dir = TemporaryDirectory()
workspace = os.path.join(temp_dir.name, "mf6Well")

In [ ]:
# Alternatively - create a folder to save the output files
workspace = os.path.join(".", "mf6Well")
try: 
    os.mkdir(workspace) 
    print("Directory '%s' created" %workspace) 
except OSError as error: 
    print(error)    

### Create the Flopy Model Objects

We are creating a square model with a specified head equal to `initial_head` along all boundaries. The head at the cells in first row in the top layer is fixed to `const_head`. First, set the name of the model and the parameters of the model: the number of layers `nlay`, the number of rows and columns `nrow` and `ncol`, lengths of the sides of the model `L`, aquifer thickness `H`, hydraulic conductivity `hk`

![conceptualModel](./images/conceptualModel.png)

In [36]:
model_name = "well_00"
initial_head = 0. # meters
const_head = 0. # meters
nlay, nrow, ncol = 1, 15, 15 
L = 15000 # meters
hk = 1.0 # m/d

One big difference between MODFLOW 6 and previous MODFLOW versions is that MODFLOW 6 is based on the concept of a simulation.  A simulation consists of the following:

* Temporal discretization (TDIS)
* One or more models (GWF is the only model supported at present)
* Zero or more exchanges (instructions for how models are coupled)
* Solutions

For this simple lake example, the simulation consists of the temporal discretization (TDIS) package (TDIS), a groundwater flow (GWF) model, and an iterative model solution (IMS), which controls how the GWF model is solved.



![mf6_simulation](./images/mf6_flowchart_p1.png)

In [37]:
# Create the Flopy simulation object
sim = flopy.mf6.MFSimulation(
    sim_name=model_name, exe_name="mf6", version="mf6", sim_ws=workspace
)


In [38]:
# Create the Flopy temporal discretization object
ntimesteps = 19
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(
    sim, pname="tdis", 
    time_units="DAYS", 
    nper=1, 
    perioddata=[(3650, ntimesteps, 1.5)],
)


In [39]:
# Create the Flopy iterative model solver (ims) Package object
ims = flopy.mf6.ModflowIms(
    sim, 
    pname="ims", 
    complexity="SIMPLE"
    )

Now that the overall simulation is set up, we can focus on building the groundwater flow model.  The groundwater flow model will be built by adding packages to it that describe the model characteristics.

!['mf6_gwf'](./images/mf6_flowchart_p2.png)

Define the discretization of the model. All layers are given equal thickness. The `bot` array is build from `H` and the `Nlay` values to indicate top and bottom of each layer, and `delrow` and `delcol` are computed from model size `L` and number of cells `N`. Once these are all computed, the Discretization file is built.

In [40]:
# Create the Flopy groundwater flow (gwf) model object
gwf_name = "gwf_well_00"
model_nam_file = "{}.nam".format(gwf_name)
gwf = flopy.mf6.ModflowGwf(
    sim, 
    modelname="gwf_well_00", 
    model_nam_file=model_nam_file
    )

In [41]:
# Create the discretization package
top, bot = 0, -100
delrow = delcol = 1000.
dis = flopy.mf6.modflow.mfgwfdis.ModflowGwfdis(
    gwf,
    pname="dis",
    length_units='METERS',
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delrow,
    delc=delcol,
    top=top,
    botm=bot,
)

In [42]:
# Create the initial conditions package
start = initial_head
ic = flopy.mf6.ModflowGwfic(
    gwf, 
    pname="ic", 
    strt=start
    )

In [43]:
# Create the node property flow package
npf = flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf(
    gwf, 
    pname="npf", 
    icelltype=1, 
    k=hk, 
    save_flows=True
)

In [44]:
# Create the storage package
sto = flopy.mf6.ModflowGwfsto(
    gwf,
    pname="sto",
    save_flows=True,
    iconvert=[0],
    ss=2.0e-6,
    sy=0.2,
    transient={0: True},
)

In [45]:
# Create the well package
tdis.nper
stress_period_data = dict()
stress_period_data[0] = [0, 7, 7,-600.,'well_00']
#stp[0] = [[],[],[]]
#stp[1] = [[],[],[],[]]


wel = flopy.mf6.ModflowGwfwel(
    gwf,
    pname="wel",
    print_input=True,
    print_flows=True,
    #auxiliary=[("var1")],
    maxbound=len(stress_period_data),
    stress_period_data=stress_period_data,
    boundnames=True,
    save_flows=True,
)

ra_wel = wel.stress_period_data.get_data(key=0)

In [46]:
# Create the constant head package.
# List information is created a bit differently for
# MODFLOW 6 than for other MODFLOW versions.  The
# cellid (layer, row, column, for a regular grid)
# must be entered as a tuple as the first entry.
# Remember that these must be zero-based indices!
chd_rec = []
for col in range(0, ncol):
    chd_rec.append([0, 0, col, const_head])

chd = flopy.mf6.ModflowGwfchd(
    gwf,
    pname="chd",
    maxbound=len(chd_rec),
    stress_period_data={0: chd_rec},
    save_flows=True,
)

In [ ]:
# The chd package stored the constant heads in a structured
# array, also called a recarray.  We can get a pointer to the
# recarray for the first stress period (iper = 0) as follows.
iper = 0
ra = chd.stress_period_data.get_data(key=0)
ra

In [ ]:
# We can make a quick plot to show where our constant
# heads are located by creating an integer array
# that starts with ones everywhere, but is assigned
# a -1 where chds are located
ibd = np.ones((nlay, nrow, ncol), dtype=int)
for k, i, j in ra["cellid"]:
    ibd[k, i, j] = -1

for k, i, j in ra_wel["cellid"]:
    ibd[k, i, j] = 0

plt.imshow(ibd[0, :, :], interpolation="none")
plt.title("Layer {}: Constant Head Cells".format(1))

In [49]:
# Create the output control package
headfile = "{}.hds".format(gwf_name)
head_filerecord = [headfile]
budgetfile = "{}.cbb".format(gwf_name)
budget_filerecord = [budgetfile]
# Note mf6 no longer supports drawdown output 
saverecord = [("HEAD", "ALL"), ("BUDGET", "ALL")]
printrecord = [("HEAD", "LAST")]
oc = flopy.mf6.modflow.mfgwfoc.ModflowGwfoc(
    gwf,
    pname="oc",
    saverecord=saverecord,
    head_filerecord=head_filerecord,
    budget_filerecord=budget_filerecord,
    printrecord=printrecord,
)

In [ ]:
# Note that help can always be found for a package
# using either forms of the following syntax
help(oc)
# help(flopy.mf6.modflow.mfgwfoc.ModflowGwfoc)

### Create the MODFLOW 6 Input Files and Run the Model

Once all the flopy objects are created, it is very easy to create all of the input files and run the model.

In [ ]:
# Write the datasets
sim.write_simulation()

In [ ]:
# Print a list of the files that were created
# in workspace
print(os.listdir(workspace))

### Run the Simulation

We can also run the simulation from the notebook, but only if the MODFLOW 6 executable is available.  The executable can be made available by putting the executable in a folder that is listed in the system path variable.  Another option is to just put a copy of the executable in the simulation folder, though this should generally be avoided.  A final option is to provide a full path to the executable when the simulation is constructed.  This would be done by specifying exe_name with the full path.

In [ ]:
# Run the simulation
success, buff = sim.run_simulation(silent=True, report=True)
if success:
    for line in buff:
        print(line)
else:
    raise ValueError("Failed to run.")